In [1]:
!pip install pyro-ppl

    100% |████████████████████████████████| 491kB 28.1MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 28.3MB/s ta 0:00:01
    79% |█████████████████████████▎      | 595.3MB 39.1MB/s eta 0:00:05  1% |▋                               | 13.4MB 38.0MB/s eta 0:00:20    2% |▊                               | 16.2MB 35.9MB/s eta 0:00:21    6% |██▏                             | 52.1MB 50.5MB/s eta 0:00:14    27% |█████████                       | 209.8MB 36.7MB/s eta 0:00:15    30% |█████████▊                      | 229.4MB 44.6MB/s eta 0:00:12    36% |███████████▋                    | 273.6MB 40.3MB/s eta 0:00:12    42% |█████████████▌                  | 317.3MB 35.5MB/s eta 0:00:13    55% |█████████████████▉              | 420.6MB 32.0MB/s eta 0:00:11    56% |██████████████████              | 424.6MB 47.0MB/s eta 0:00:07    56% |██████████████████▏             | 426.9MB 27.7MB/s eta 0:00:12    66% |█████████████████████▏          | 498.4MB 37.8MB/s eta 0:00:07

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



    81% |██████████████████████████▏     | 616.6MB 42.3MB/s eta 0:00:04

    100% |████████████████████████████████| 753.4MB 16kB/s  eta 0:00:01    89% |████████████████████████████▊   | 674.9MB 34.9MB/s eta 0:00:03    89% |████████████████████████████▊   | 677.1MB 32.6MB/s eta 0:00:03    91% |█████████████████████████████▏  | 686.1MB 40.9MB/s eta 0:00:02
  Running setup.py bdist_wheel for opt-einsum ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/2c/b1/94/43d03e130b929aae7ba3f8d15cbd7bc0d1cb5bb38a5c721833
Successfully built opt-einsum
fastai 1.0.59 requires nvidia-ml-py3, which is not installed.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
%matplotlib inline
# python libraties
import os, itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
from PIL import Image

# pytorch libraries
import torch
from torch import optim,nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
from torchvision import models,transforms

# sklearn libraries
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
# data directory 
data_dir = 'data'   

In [4]:
def set_parameter_requires_grad(model, finetuning = False):
    '''
        This function freezes the parameters of the model.
        
        Parameters:
            model: model to be freezed before training
            finetuning: if finetuning is true then the model will not be freezed
        
        Returns:
            Nothing
    '''
    if not finetuning:
        for param in model.parameters():
            param.requires_grad = False

In [5]:
class FCModel(nn.Module):
    def __init__(self, num_ftrs, hidden_ftrs1, hidden_ftrs2, output_ftrs):
        
        super(FCModel, self).__init__()
        self.fc1 = nn.Linear(num_ftrs, hidden_ftrs1)
        self.fc2 = nn.Linear(hidden_ftrs1, hidden_ftrs2)
        self.fc3 = nn.Linear(hidden_ftrs2, output_ftrs)
        self.drop1 = nn.Dropout(0.3)
        self.drop2 = nn.Dropout(0.3)
        
        self.softmax = nn.Softmax()
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = F.relu(self.fc2(x))
        x = self.drop(x)
        x = self.fc3(x)
        return self.softmax(x)
        

In [6]:
def initialize_models(model_name, num_classes, fine_tuning = False):
    '''
        This function initialize3s the pretrained model
    '''
    
    model_fe = None
    input_size = 0
    
    if(model_name == 'resnet'):
        model_fe = models.resnet50(pretrained = True)
        set_parameter_requires_grad(model_fe, fine_tuning)
        num_ftrs = model_fe.fc.in_features
        hidden_ftrs1 = model_fe.fc.out_features
        hidden_ftrs2 = int(hidden_ftrs1 / 2)
        model_fe.fc = FCModel(num_ftrs, hidden_ftrs1, hidden_ftrs2, num_classes)
        input_size = 224
    
    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_fe, input_size
        

In [7]:
model, input_size = initialize_models('resnet', num_classes = 7)
print(input_size)
model

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/ec2-user/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 106MB/s]


224


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
mean = np.load('means.npz')
stdevs = np.load('stdevs.npz')
norm_mean = mean['arr_0']
norm_std = stdevs['arr_0']
print(norm_mean)
print(norm_std)

[0.7630329  0.54564583 0.5700466 ]
[0.14092815 0.15261224 0.1699708 ]


In [9]:
# transfromations for train images
train_transform = transforms.Compose([transforms.Resize((input_size,input_size)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomRotation(20),
                                      transforms.ColorJitter(brightness=0.1, contrast=0.1, hue=0.1),
                                      transforms.ToTensor(), 
                                      transforms.Normalize(norm_mean, norm_std)])
# define the transformation of the test images.
test_transform = transforms.Compose([transforms.Resize((input_size,input_size)), 
                                    transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])

In [15]:
class HamDataset(Dataset):
    def __init__(self, csvpath, transform = None):
        self.df = pd.read_csv(csvpath)
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        X = Image.open(self.df['path'][index])
        y = torch.tensor(int(self.df['cell_type_idx'][index]))
        
        if self.transfrom:
            X = self.transform(X)
        
        return X, y

def get_data_loader(csvpath, transform = test_transform, batch_size = 32):
    if csvpath == 'train.csv':
        transform = train_transform
    csvpath = os.path.join(data_dir, csvpath)
    dataset = HamDataset(csvpath, transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    
    return dataloader

In [18]:
train_dl = get_data_loader('train.csv')

In [24]:
len(train_dl)*32

28832

False